In [3]:
"""
# 전처리 배분

1(성준) - 0~9,   40~49, 80~89, 120, 
       - credit_card_balance.csv
       - POS_CASH_balance.csv
       - previous_application.csv
       - installments_payments.csv
2(장호) - 10~19, 50~59, 90~99, 121
3(두영) - 20~29, 60~69, 100~109, 122, 
       - bureau.csv,
4(혜빈) - 30~39, 70~79, 110~119, 123, 
       - bureau_balance.csv
5(준혁) -  
"""

"""
# 데이터 목록
application_test.csv
application_train.csv
credit_card_balance.csv
POS_CASH_balance.csv
previous_application.csv
installments_payments.csv
bureau.csv
bureau_balance.csv
sample_submission.csv
"""

'\n# 데이터 목록\napplication_test.csv\napplication_train.csv\ncredit_card_balance.csv\nPOS_CASH_balance.csv\nprevious_application.csv\ninstallments_payments.csv\nbureau.csv\nbureau_balance.csv\nsample_submission.csv\n'

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
import user_functions as uf

%matplotlib inline
# 한글 출력되게 설정
uf.korean2()

In [2]:
# Load file
train_datasets = pd.read_csv(uf.get_f_path()+"application_train.csv")
test_datasets = pd.read_csv(uf.get_f_path()+"application_test.csv")

# Label 만들기
train_target_ds = train_datasets["TARGET"]

# TARGET feature 삭제
train_datasets.drop(columns="TARGET", inplace=True)

In [ ]:
# # info
# print(train_datasets.info())
# print(test_datasets.info())

# # train은 307,511 행
# # test는   48,744 행
# # -----------------
# # 총      356,255 행

In [3]:
# train datasets과 test datasets을 구분하는 "train/test" feature 생성
train_datasets["train/test"] = "train"
test_datasets["train/test"] = "test"

# 합치기
train_test_datasets = pd.concat([train_datasets, test_datasets], ignore_index=True)

# 356,255 행인지 확인
train_test_datasets.info()

# index 재설정
# print(train_datasets.index)
# print(test_datasets.index)
train_test_datasets.reset_index(drop=True, inplace=True)
# print(train_test_datasets.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 122 entries, SK_ID_CURR to train/test
dtypes: float64(65), int64(40), object(17)
memory usage: 331.6+ MB


In [95]:
# 전처리 데이터
prepro_ds = train_test_datasets.copy()

In [96]:
##########################################################
######################## SUNGJUNE ########################
##########################################################
# 자녀가 5명 이상인 값을 5로 처리
prepro_ds.loc[prepro_ds["CNT_CHILDREN"] >= 5, "CNT_CHILDREN"] = 5

# 이상치 117000000에서 0을 하나 제거함
prepro_ds.loc[prepro_ds["AMT_INCOME_TOTAL"] > 1e+08, "AMT_INCOME_TOTAL"] = prepro_ds.loc[prepro_ds["AMT_INCOME_TOTAL"] > 1e+08, "AMT_INCOME_TOTAL"].map(lambda x: x / 10)

# "AMT_ANNUITY"를 "AMT_CREDIT"기준으로 가장 유사한 값을 통해 대체함
prepro_ds["AMT_ANNUITY"] = \
    uf.imput_other_f_mean(prepro_ds, "AMT_ANNUITY", "AMT_CREDIT")

# "AMT_GOODS_PRICE"를 "AMT_CREDIT"기준으로 가장 유사한 값을 통해 대체함
prepro_ds["AMT_GOODS_PRICE"] = \
    uf.imput_other_f_mean(prepro_ds, "AMT_GOODS_PRICE", "AMT_CREDIT")

# SimpleImputer 사용
# sp_input = SimpleImputer()
# prepro_ds["AMT_GOODS_PRICE"] = \
#     pd.DataFrame(sp_input.fit_transform(\
#         prepro_ds[["AMT_GOODS_PRICE", "AMT_CREDIT"]]
#         ,columns=["AMT_GOODS_PRICE", "AMT_CREDIT"]
#                                        )
#                 )["AMT_GOODS_PRICE"].hist()

# "EXT_SOURCE_1"의 결측치를 0으로 대체함
# 외부 자료의 신뢰성을 의미하는 자료이므로 외부 자료가 없는 것을 의미한다고 판단됨
prepro_ds["EXT_SOURCE_1"].fillna(0, inplace=True)
prepro_ds["EXT_SOURCE_2"].fillna(0, inplace=True)
prepro_ds["EXT_SOURCE_3"].fillna(0, inplace=True)
prepro_ds["FONDKAPREMONT_MODE"].fillna("not specified", inplace=True)

# 결측치를 0으로 대체함
zero_imputation_lists= ["FLOORSMIN_MEDI",
                        "LANDAREA_MEDI",
                        "LIVINGAPARTMENTS_MEDI",
                        "LIVINGAREA_MEDI",
                        "NONLIVINGAPARTMENTS_MEDI",
                        "NONLIVINGAREA_MEDI",
                        "TOTALAREA_MODE",
                        "APARTMENTS_AVG",
                        "BASEMENTAREA_AVG",
                        "YEARS_BEGINEXPLUATATION_AVG",
                        "YEARS_BUILD_AVG",
                        "COMMONAREA_AVG",
                        "ELEVATORS_AVG"]
for feature in zero_imputation_lists:
    prepro_ds.loc[:, feature].fillna(0, inplace=True)

# 중앙값으로 대체함
imput_mean = SimpleImputer(strategy="median")
prepro_ds["ENTRANCES_AVG"] = imput_mean.fit_transform(prepro_ds[["ENTRANCES_AVG"]])

# "WALLSMATERIAL_MODE" "Others"로 대체
prepro_ds["WALLSMATERIAL_MODE"].fillna("Others", inplace=True)

# 이상치를 정상값 범위 중 가장 큰 8로 대체함
prepro_ds["AMT_REQ_CREDIT_BUREAU_QRT"].fillna(8, inplace=True)

# "HOUSETYPE_MODE" 결측치 "XNA"로 대체 ∵ HOUSE외 주거 시설에 거주한다고 판단.
prepro_ds["HOUSETYPE_MODE"].fillna("XNA", inplace=True)


In [98]:
##########################################################
######################### JANGHO #########################
##########################################################
# Null 처리
# prepro_ds['AMT_GOODS_PRICE'].fillna(prepro_ds['AMT_GOODS_PRICE'].mean(), inplace = True)
prepro_ds['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace = True)
prepro_ds['EMERGENCYSTATE_MODE'].fillna('No', inplace = True)
prepro_ds['AMT_REQ_CREDIT_BUREAU_YEAR'].fillna(0, inplace = True)
prepro_ds.loc[:, "ENTRANCES_AVG":"BASEMENTAREA_MODE"] = \
    prepro_ds.loc[:, "ENTRANCES_AVG":"BASEMENTAREA_MODE"].fillna(0)
prepro_ds.loc[:, "EMERGENCYSTATE_MODE":"DAYS_LAST_PHONE_CHANGE"] = \
    prepro_ds.loc[:, "EMERGENCYSTATE_MODE":"DAYS_LAST_PHONE_CHANGE"].fillna(0)
prepro_ds.loc[:, "AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(0, inplace=True)

# IQR을 이용한 Outlier median 대체
def replace_outlier_by_med(feature):
    q1, q3 = np.percentile(feature, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr*1.5)
    upper_bound = q3 + (iqr*1.5)
    
    mask = np.where((feature > upper_bound) | (feature < lower_bound),feature.median(), feature)
    return mask

prepro_ds['AMT_GOODS_PRICE'] = replace_outlier_by_med(prepro_ds['AMT_GOODS_PRICE'])

In [100]:
##########################################################
######################### HYEBIN #########################
##########################################################
prepro_ds.loc[:, "NONLIVINGAPARTMENTS_MODE":"FLOORSMAX_MEDI"] = \
    prepro_ds.loc[:, "NONLIVINGAPARTMENTS_MODE":"FLOORSMAX_MEDI"].fillna(0)
prepro_ds.loc[:, "AMT_REQ_CREDIT_BUREAU_HOUR":"AMT_REQ_CREDIT_BUREAU_MON"] = \
prepro_ds.loc[:, "AMT_REQ_CREDIT_BUREAU_HOUR":"AMT_REQ_CREDIT_BUREAU_MON"].fillna(0)

In [101]:
##########################################################
######################## DOOYOUNG ########################
##########################################################
prepro_ds["OWN_CAR_AGE"].fillna(-1, inplace=True)
prepro_ds.loc[:, "YEARS_BEGINEXPLUATATION_MODE":"LIVINGAREA_MODE"] = \
    prepro_ds.loc[:, "YEARS_BEGINEXPLUATATION_MODE":"LIVINGAREA_MODE"].fillna(0)
prepro_ds["OCCUPATION_TYPE"].fillna("Inoccupation", inplace=True)

prepro_ds.loc[prepro_ds["CNT_FAM_MEMBERS"] >= 8, "CNT_FAM_MEMBERS"] = 8
prepro_ds["CNT_FAM_MEMBERS"].fillna(1, inplace=True)

In [102]:
uf.check_missing_value(prepro_ds)

결측치 없음!


---
---

In [103]:
credit_card_ds = \
    pd.read_csv("../pre_data/new_credit_card_ds_balance.csv")
instal_pay_ds = \
    pd.read_csv("../pre_data/new_installments_payments.csv")
pos_cash_ds = \
    pd.read_csv("../pre_data/new_POS_CASH_balance.csv")
prev_app_ds = \
    pd.read_csv("../pre_data/new_previous_application.csv")
bureau_final = \
    pd.read_csv("../pre_data/bureau_final.csv")

In [106]:
# feature 중복 값 확인

colum_list = []
# colum_list.append(list(credit_card_ds.columns))
colum_list.append(list(instal_pay_ds.columns))
colum_list.append(list(pos_cash_ds.columns))
colum_list.append(list(prev_app_ds.columns))
colum_list.append(list(prepro_ds.columns))
colum_list.append(list(bureau_final.columns))

col_list = []
for a in colum_list:
    for b in a:
        col_list.append(b)

pd.DataFrame(col_list).value_counts()#.unique()

SK_ID_CURR                     5
AMT_ANNUITY                    2
LIVINGAREA_MODE                1
LIVE_REGION_NOT_WORK_REGION    1
LIVINGAPARTMENTS_AVG           1
                              ..
EMERGENCYSTATE_MODE            1
ELEVATORS_MODE                 1
ELEVATORS_MEDI                 1
ELEVATORS_AVG                  1
연체일                            1
Length: 138, dtype: int64

In [109]:
# feature name change
# credit_card_ds.rename(columns={"SK_DPD": "연체기간(credit_c)",
#                                "AMT_RECEIVABLE_PRINCIPAL": "미수원금",
#                                "소액연체_비율": "소액연체비중(credit_c)"},
#                       inplace=True
#                      )
pos_cash_ds.rename(columns={"SK_DPD": "연체기간(pos_cash)",
                            "소액연체_비율": "소액연체비중(pos_cash)"},
                   inplace=True
                  )
bureau_final.rename(columns={"AMT_ANNUITY": "AMT_ANNUITY(bure)"},
                    inplace=True
                   )

In [111]:
# credit_card.csv의 경우 결측값이 252,697개에 달해 사용할 수 없다고 판단됨.
# merge_ds = pd.merge(prepro_ds, credit_card_ds, on="SK_ID_CURR", how="left")
merge_ds = pd.merge(prepro_ds, prev_app_ds, on="SK_ID_CURR", how="left")
merge_ds = pd.merge(merge_ds, instal_pay_ds, on="SK_ID_CURR", how="left")
merge_ds = pd.merge(merge_ds, pos_cash_ds, on="SK_ID_CURR", how="left")
merge_ds = pd.merge(merge_ds, bureau_final, on="SK_ID_CURR", how="left")

In [112]:
temp_ds = merge_ds[["train/test", "연체일"]]
temp_ds.loc[temp_ds["연체일"].isna()]["train/test"].value_counts()

train    33460
test      3486
Name: train/test, dtype: int64

In [113]:
uf.check_missing_value(merge_ds)

122        승인/거부(prev_app)                36,946
123            (추정)이자율                    36,946
124              연체일                      36,946
125             연체여부                      36,946
126            승인액/요청액                    36,946
127           과거_상환액/총액                   16,671
128        연체기간(pos_cash)                 19,003
129       소액연체비중(pos_cash)                19,003
130       CREDIT_ACTIVE_idx               50,444
131        CREDIT_TYPE_idx                50,444
132      CREDIT_DAY_OVERDUE               50,444
133    AMT_CREDIT_MAX_OVERDUE             50,444
134        AMT_CREDIT_SUM                 50,444
135      AMT_CREDIT_SUM_DEBT              50,444
136    AMT_CREDIT_SUM_OVERDUE             50,444
137       AMT_ANNUITY(bure)               50,444
138        balance_STATUS                 50,444
